Original DataFrame:
      Name  Age         City
0    Alice   25     New York
1      Bob   30  Los Angeles
2  Charlie   35      Chicago
3    David   40      Houston
4     Ella   45      Phoenix

Filtered DataFrame (Age > 30):
      Name  Age     City  Age Next Year
2  Charlie   35  Chicago             36
3    David   40  Houston             41
4     Ella   45  Phoenix             46

Average Age: 35.0
